# Pre-processing of the Dataset

In [1]:
import os
from PIL import Image
import cv2
import numpy as np

In [2]:
# Path of dataset
path = "Data"

## Cropping sky from the image

In [5]:
count = 0
for folder in os.listdir(path):
    os.mkdir("Cropped_data" + '/' + folder)
    path_ = path + "/" +folder
    for file in os.listdir(path_):
        count += 1
        image_path = path_ + "/" + file
        img = Image.open(image_path)
        
        # Calculating edges in the image
        if img.size[0] >= img.size[1]:
            read_image = cv2.imread(image_path, 50)
            edges = cv2.Canny(read_image, 150, 300)
            shape = np.shape(edges)
            left = np.sum(edges[0:shape[0] // 2, 0:shape[1] // 2])
            right = np.sum(edges[0:shape[0] // 2, shape[1] // 2:])
            
            # More edges = Building
            # Less edges = Sky
            if right > left:
                sky_side = 0
            else:
                sky_side = 1
                
            # Resizing image to a particular size
            base_height = 400
            wpercent = (base_height / float(img.size[1]))
            wsize = int((float(img.size[0]) * float(wpercent)))
            img = img.resize((wsize, base_height), Image.ANTIALIAS)
                
            #Cropping sky area from the image
            if sky_side == 0:
                img = img.crop((0, 0, base_height, img.size[1])) 
            else: 
                img = img.crop((img.size[0]-base_height, 0, img.size[0], img.size[1]))
                    
            # Saving the cropped image
            destination = "Cropped_data" + "/" + folder + "/" + file
            img.save(destination)
                
        else:
            base_width = 400
            wpercent = (base_width / float(img.size[0]))
            hsize = int((float(img.size[1]) * float(wpercent)))
        
            # Resizing the image
            img = img.resize((base_width, hsize), Image.ANTIALIAS)
                
            # Cropping the image
            img = img.crop((0, 0, img.size[0], 400))
                
            # Saving the cropped image
            destination = "Cropped_data" + "/" + folder + "/" + file
            img.save(destination)
                
print ("----------Total no. of images cropped: ", count, "----------")

----------Total no. of images cropped:  4621 ----------


## Converting images to matrix

In [6]:
from keras.preprocessing import image as image_utils
import PIL.ImageOps
import random

In [9]:
# Path of cropped image
image_root = "Cropped_data/"

# 200 sized model
batch_size_for_models = 2
    
train_data = []
train_label = []
    
classes_dir = os.listdir(image_root)
    
fc = 0
    
for cls in classes_dir:
    counter = 0
    class_list = os.listdir(image_root + cls + "/")
    for imagename in class_list:
        counter +=1
            
        # Converting image to array
        img = image_utils.load_img(image_root + cls + "/" + imagename, target_size = (100,100))
        img = PIL.ImageOps.invert(img)
        img = image_utils.img_to_array(img)
            
        # Appending array to the list
        train_data.append(img)
        train_label.append(int(cls))
    
        # If batch size if completed, Save model
        if counter == batch_size_for_models:
            # Saving our model
            dest = "Data_npy/"
            # Image data
            np.save(dest + "train_data" + ".npy", np.array(train_data))
            # Image labels
            np.save(dest + "train_label" + ".npy",np.array(train_label))
             
                
# Add the images which are left
if (len(train_data) != 0):
    np.save(dest+ "train_data.npy", np.array(train_data))
    np.save(dest+ "train_label.npy", np.array(train_label))